# TF-IDF implementation

## Read documents 

In [1]:
# Auto reload .py imports
%load_ext autoreload
%autoreload 2

In [2]:
import utils
from collections import defaultdict
from tf_idf_utils import compute_unique_words, compute_tf
from concurrent.futures import ProcessPoolExecutor
import numpy as np
from tqdm import tqdm

IDF_SAVE_PATH = './.cache/idf_scores.pkl'
TF_IDF_SAVE_PATH = './.cache/tf_idf_docs.pkl'

idf  = defaultdict(float)
tf_docs = defaultdict(list)
total_docs = 0
with ProcessPoolExecutor() as executor:
    for batch in tqdm(utils.batch_load_documents(executor)):
        total_docs += len(batch)
        
        list_unique_words =  compute_unique_words(batch)

        for unique_words in list_unique_words:
            for unique_word in unique_words:
                idf[unique_word] += 1
        
        tfs_docs_id = list(executor.map(compute_tf, batch, chunksize=utils.LOAD_BATCH_SIZE // 32))
        for doc_id, tf_doc in tfs_docs_id:
            for word, tf in tf_doc.items():
                if word not in tf_docs:
                    tf_docs[word] = [(doc_id, tf)]
                else:
                    tf_docs[word].append((doc_id, tf))
        print(f"Processed {total_docs} documents", end='\r')
        

idf = {word: np.log(total_docs / (1 + idf[word])) for word in idf}

utils.save(IDF_SAVE_PATH, idf)

# tf_docs = {doc_id: {word: tf * idf[word] for word, tf in tf_doc.items()} for doc_id, tf_doc in tf_docs.items()}
# utils.save(TF_IDF_SAVE_PATH, tf_docs)




    

0it [00:00, ?it/s]

Loaded 268022 documents.


1it [01:53, 113.98s/it]

2it [02:37, 72.58s/it] 

3it [03:29, 62.97s/it]

4it [04:20, 58.51s/it]

In [4]:
assert all(isinstance(doc, utils.Document) for doc in documents)
print(type(documents))
print(type(documents[0]))



<class 'list'>
<class 'utils.Document'>


In [5]:
for doc in documents:
    doc.tokenize()
    del doc.content

In [4]:
from tf_idf_utils import compute_tfidf

tfidf = compute_tfidf(documents)